In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

import catboost as cb
import eli5

2023-04-02 15:40:10.660253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 15:40:10.808761: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-02 15:40:10.808782: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-02 15:40:10.840652: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-02 15:40:11.742661: W tensorflow/stream_executor/platform/de

In [2]:
# Load files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')

In [3]:
# identify target
target_cols = []
for i in train.columns.values:
    if i not in test.columns.values:
        target_cols.append(i)
        
target = train[target_cols]
train[target_cols].head()

,target
0,51.500000
1,55.869999
2,47.520000
3,58.599998
4,76.599998


In [4]:
# select features according to null values
# train = train[features]
# test  = test[features[:-1]]

In [5]:
# select features according to correlatin to target

# get the correlation of each feature with respect to the target (emmission)
train_corr = train.corr()['target'].abs().sort_values(ascending=False)

# get features with a correlation above a certain threshold
features = train_corr[train_corr > 0].index.to_list()[1:]

print(len(features))

train = train[features]
test  = test[features]

# train['target'] = target

159


In [6]:
# max(train['target']), min(train['target'])

In [7]:
train = train.fillna('')
test  = test.fillna('')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    train,
    target,
    test_size = 0.15,
    random_state = 42,
    shuffle = True
)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7297, 159), (1288, 159), (7297, 1), (1288, 1))

In [10]:
# y_test = test[features[:-1]]
train_dataset = cb.Pool(data = X_train, label = y_train)
test_dataset   = cb.Pool(data = X_test, label = y_test)
model         = cb.CatBoostRegressor(random_seed=123, verbose=300)
model.fit(train_dataset, eval_set=test_dataset, use_best_model=True, early_stopping_rounds=300)
y_pred = model.predict(X_test)
# preds_test  = model.predict(test)
print(np.sqrt(mean_squared_error(y_test, y_pred)))

Learning rate set to 0.069568
0:	learn: 14.7557798	test: 15.2609053	best: 15.2609053 (0)	total: 59.4ms	remaining: 59.4s
300:	learn: 9.0676740	test: 10.4526690	best: 10.4526690 (300)	total: 5.37s	remaining: 12.5s
600:	learn: 8.0419387	test: 10.1677345	best: 10.1670744 (599)	total: 8.79s	remaining: 5.83s
900:	learn: 7.2829808	test: 10.0629962	best: 10.0625402 (887)	total: 12.5s	remaining: 1.38s
999:	learn: 7.0713188	test: 10.0350993	best: 10.0343640 (998)	total: 13.6s	remaining: 0us

bestTest = 10.03436404
bestIteration = 998

Shrink model to first 999 iterations.
10.034364045607942


In [11]:
import shap
expaliner = shap.TreeExplainer(model)
shap_values = expaliner.shap_values(test)

In [12]:
# features = X_test.columns
# len(f['f1'])

In [13]:
f = {
    'f1' : [], 'f2': [], 'f3': [], 'f4': [],'f5': [],
    'f6' : [], 'f7': [], 'f8': [], 'f9': [],'f10': [],
    'f11' : [], 'f12': [], 'f13': [], 'f14': [],'f15': []
}

In [14]:
for shap_value in shap_values:
    arr = np.argsort(shap_value)[::-1][:15]
    for ind, a in enumerate(arr):
        name_f = f'f{ind+1}'
        f[name_f].append(features[a])

In [15]:
# Make predictions on the test set
pred = model.predict(test)
pred

array([55.92092927, 46.96785268, 52.38558481, ..., 41.00516655,
       43.21949268, 44.59174206])

In [16]:
SampleSubmission['target']    = pred
SampleSubmission['feature_1'] = f['f1']
SampleSubmission['feature_2'] = f['f2']
SampleSubmission['feature_3'] = f['f3']
SampleSubmission['feature_4'] = f['f4']
SampleSubmission['feature_5'] = f['f5']
SampleSubmission['feature_6'] = f['f6']
SampleSubmission['feature_7'] = f['f7']
SampleSubmission['feature_8'] = f['f8']
SampleSubmission['feature_9'] = f['f9']
SampleSubmission['feature_10'] = f['f10']
SampleSubmission['feature_11'] = f['f11']
SampleSubmission['feature_12'] = f['f12']
SampleSubmission['feature_13'] = f['f13']
SampleSubmission['feature_14'] = f['f14']
SampleSubmission['feature_15'] = f['f15']

In [17]:
SampleSubmission

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,ID_0I0999N6S,55.920929,child_observe_total,teacher_emotional_total,pri_fees_amount_pv,count_children_present,count_staff_qual_skills,pri_fees_amount,child_zha,ses_proxy,id_team,id_facility_n,count_toilets_children,id_mn_n,count_register_year_2018,count_register_gender_female,child_attendance
1,ID_GQ6ONJ4FP,46.967853,child_observe_total,teacher_emotional_total,count_register_year_2015,count_children_present,pri_fees_amount_pv,count_children_precovid,id_facility,count_register_year_2020,pri_fees_amount,id_facility_n,count_register_year_grader,quintile,count_staff_qual_skills,count_register_race_coloured,id_mn_n
2,ID_YZ76CVRW3,52.385585,id_enumerator,child_observe_total,longitude,id_team,teacher_selfcare_total,pri_fees_amount_pv,id_prov_n,count_register_year_2019,latitude,count_register_race,pri_fees_amount,count_register_year_2017,ses_proxy,obs_classrooms,count_register_year_2018
3,ID_BNINCRXH8,68.001217,child_observe_total,id_enumerator,child_age,teacher_emotional_total,teacher_duration,id_prov_n,count_children_present,id_team,child_attendance,obs_heating_4,count_children_attendance,pri_funding_subsidy,count_register_gender_female,data_year,pri_capacity
4,ID_1U7GDTLRI,43.644044,child_observe_total,id_ward,id_team,longitude,child_months_enrolment,latitude,count_register_year_2020,id_facility,count_register_gender_female,count_children_precovid,pri_time_close_hours,pri_year,id_mn_n,count_staff_gender_other,pqa_score_environment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,52.102360,child_age,child_observe_total,id_ward,ses_proxy,teacher_selfcare_total,id_team,count_register_race,pri_funding_subsidy,pri_time_open_minutes,data_year,count_register_year_2020,count_register_year_2017,ward_best,obs_classrooms,id_dc_n
3676,ID_H2RKKMMKK,50.943590,child_age,id_enumerator,id_ward,ward_best,id_team,longitude,pri_fees_amount_pv,count_children_present,child_zha,child_height,count_register_year_2019,teacher_selfcare_total,pri_fees_amount,count_register_year_2020,count_children_precovid
3677,ID_VY8KX7YTZ,41.005167,child_observe_total,child_age,count_register_gender_female,teacher_social_total,obs_cooking_2,id_facility_n,count_register_year_2020,pri_time_close_hours,count_children_precovid,count_register_year_2018,pqa_score_curriculum,pri_year,pra_class_size,count_practitioners_age_3,count_staff_time_full
3678,ID_EO2MYZ4M7,43.219493,id_enumerator,obs_heating_1,count_staff_qual_skills,child_zha,id_facility,pqa_score_assessment,id_dc_n,pqa_score_relationships,latitude,pri_expense_food,pri_capacity,count_children_present,count_register_year_2018,count_register_year_2020,child_height


In [19]:
SampleSubmission.to_csv('submission.csv', index=False)
